## Since we compare the performance of our approach with the NMS, soft-NMS, and WBF, you must install ensemble-boxes library.

In [1]:
!pip install ensemble-boxes

In [2]:
import os
import numpy as np
from tqdm import tqdm
from pathlib import Path
from utils.metrics import create_plots
from utils.consensus_WBF import calculate_consensus_focus, ensemble_reweight, weighted_boxes_fusion, convert_darknet, toDarknet
from ensemble_boxes import *

In [3]:
m_folder="inputs/labels"
data_files = [x[0] for x in os.walk(m_folder)] #folder list
m1_folder = data_files[1:][0]+'/'
data_files1 = [x[2] for x in os.walk(m1_folder)][0] #list of txt files, one per image
boxes_lists = []
scores_lists = []
labels_lists = []
for i in range(len(data_files1)):
    boxes_list = []
    scores_list = []
    labels_list = []
    for j in range(1,len(data_files)):
        tempo=open(data_files[j]+'/'+data_files1[i]).read().split('\n')
        lbl_list, box_list, score_list=[], [], []
        if len(tempo[0])!=0:
            for k in range(len(tempo)):
                if tempo[k]!='':
                    temporal=tempo[k].split(' ')
                    lbl_list.append(int(temporal[0]))
                    box_list.append([float(u) for u in temporal[1:5]])
                    score_list.append(float(temporal[5]))
        else:
            lbl_list.append(0)
            box_list.append([0.0, 0.0, 0.0, 0.0])
            score_list.append(0)
        boxes_list.append(box_list)
        scores_list.append(score_list)
        labels_list.append(lbl_list)
    boxes_lists.append(boxes_list)
    scores_lists.append(scores_list)
    labels_lists.append(labels_list)

In [4]:
def transform_box(lists):
    #Input -> YOLOv5 format: f"{category_idx} {x1 + bbox_width / 2} {y1 + bbox_height / 2} {bbox_width} {bbox_height}\n"
    #Output -> Coordinates for boxes expected to be normalized e.g in range [0; 1]. Order: x1, y1, x2, y2.
    bxs_lists = []
    for l in range(len(lists)):
        b_list = []
        for m in range(len(lists[l])):
            if not all(r == 0 for r in res):
                bbox_width = lists[l][m][2]
                bbox_height = lists[l][m][3]
                x1 = lists[l][m][0] - bbox_width / 2 
                y1 = lists[l][m][1] - bbox_height / 2 
                x2 = bbox_width + x1
                y2 = bbox_height + y1
            else:
                x1,y1,x2,y2=0,0,0,0
            b_list.append([float(np.clip(x1,0,1)), float(np.clip(y1,0,1)), float(np.clip(x2,0,1)), float(np.clip(y2,0,1))])
        bxs_lists.append(b_list)
    return bxs_lists

def transform_yolo(lists):
    #Input -> Coordinates for boxes expected to be normalized e.g in range [0; 1]. Order: x1, y1, x2, y2.
    #Output -> YOLOv5 format: f"{category_idx} {x1 + bbox_width / 2} {y1 + bbox_height / 2} {bbox_width} {bbox_height}\n"
    bxs_lists = []
    for l in range(len(lists)):
        x1,y1=lists[l][0],lists[l][1]
        x2,y2=lists[l][2],lists[l][3]
        bbox_width = x2 - x1
        bbox_height = y2 - y1
        cx = x1 + bbox_width / 2
        cy = y1 + bbox_height / 2
        bxs_lists.append([float(np.clip(cx,0,1)), float(np.clip(cy,0,1)), float(np.clip(bbox_width,0,1)), float(np.clip(bbox_height,0,1))])
    return bxs_lists

In [5]:
weights = [1,1,1,1]
iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1

In [6]:
newlbl_dir='output/labels_nms/'
labels_path = Path(f"{newlbl_dir}")#labels path
if not os.path.exists(labels_path):
    os.makedirs(labels_path)
for i in tqdm(range(len(data_files1))):
    res = [x for y in boxes_lists[i] for x in y] 
    res = [x for y in res for x in y]
    element=Path(labels_path,data_files1[i])
    f = open(Path(element),mode="w")
    if not all(r == 0 for r in res):
        bxs_lists=transform_box(boxes_lists[i])
        bxs, scores, labels = nms(bxs_lists, scores_lists[i], labels_lists[i], weights=weights, iou_thr=iou_thr)
        boxes = transform_yolo(bxs)
        for j in range(len(boxes)):
            if float(scores[j]) >= skip_box_thr:
                f.write(f"{int(labels[j])} {boxes[j][0]} {boxes[j][1]} {boxes[j][2]} {boxes[j][3]} {scores[j]}\n")
            else:
                f.write("\n")
    else:
        f.write("\n")
    f.close()

  0%|                                                                                          | 0/928 [00:00<?, ?it/s]

Warning. Removed 1 boxes with zero area!


  6%|█████                                                                            | 58/928 [00:03<00:35, 24.35it/s]

Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed

 19%|███████████████▏                                                                | 176/928 [00:03<00:07, 95.04it/s]

...

Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed

 85%|███████████████████████████████████████████████████████████████████▎           | 790/928 [00:04<00:00, 545.12it/s]

Warning. Removed 1 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed

100%|███████████████████████████████████████████████████████████████████████████████| 928/928 [00:04<00:00, 192.70it/s]

Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed

In [7]:
newlbl_dir='output/labels_soft_nms/'
labels_path = Path(f"{newlbl_dir}")#labels path
if not os.path.exists(labels_path):
    os.makedirs(labels_path)
for i in tqdm(range(len(data_files1))):
    res = [x for y in boxes_lists[i] for x in y] 
    res = [x for y in res for x in y]
    element=Path(labels_path,data_files1[i])
    f = open(Path(element),mode="w")
    if not all(r == 0 for r in res):
        bxs_lists=transform_box(boxes_lists[i])
        bxs, scores, labels = soft_nms(bxs_lists, scores_lists[i], labels_lists[i], iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
        boxes = transform_yolo(bxs)
        for j in range(len(boxes)):
            f.write(f"{int(labels[j])} {boxes[j][0]} {boxes[j][1]} {boxes[j][2]} {boxes[j][3]} {scores[j]}\n")
    else:
        f.write("\n")
    f.close()

  0%|                                                                                          | 0/928 [00:00<?, ?it/s]

Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!


  4%|███▌                                                                            | 41/928 [00:00<00:02, 403.05it/s]

Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed

  9%|███████▍                                                                        | 86/928 [00:00<00:01, 424.47it/s]

...

Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed

 97%|████████████████████████████████████████████████████████████████████████████▊  | 903/928 [00:01<00:00, 500.43it/s]

Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed

100%|███████████████████████████████████████████████████████████████████████████████| 928/928 [00:02<00:00, 457.68it/s]

Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 3 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!


In [8]:
newlbl_dir='output/labels_wbf_all_1/'
labels_path = Path(f"{newlbl_dir}")#labels path
if not os.path.exists(labels_path):
    os.makedirs(labels_path)
for i in tqdm(range(len(data_files1))):
    res = [x for y in boxes_lists[i] for x in y] 
    res = [x for y in res for x in y]
    element=Path(labels_path,data_files1[i])
    f = open(Path(element),mode="w")
    if not all(r == 0 for r in res):
        bxs_lists=transform_box(boxes_lists[i])
        bxs, scores, labels = weighted_boxes_fusion(bxs_lists, scores_lists[i], labels_lists[i], weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes = transform_yolo(bxs)
        for j in range(len(boxes)):
            f.write(f"{int(labels[j])} {boxes[j][0]} {boxes[j][1]} {boxes[j][2]} {boxes[j][3]} {scores[j]}\n")
    else:
        f.write("\n")
    f.close()

100%|███████████████████████████████████████████████████████████████████████████████| 928/928 [00:01<00:00, 478.60it/s]
